In [38]:
import tensorflow as tf

In [39]:
import matplotlib.pyplot as plt
%matplotlib inline

In [41]:
from tensorflow.examples.tutorials.mnist import input_data

In [42]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [43]:
# Initialize weights

In [44]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)

In [45]:
# Initialize bias

In [46]:
def init_bias(shape):
    init_bias_dist = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_dist)

In [47]:
# Conv2d

In [48]:
def conv2d(x,W):
    # x --> [batch,H,W,Channels]
    # W --> [H,W,Channels in, Channels out]
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

In [49]:
# Pooling

In [50]:
def max_pooling(x):
    # x --> [batch,H,W,Channels]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [51]:
# Convolutional Layer
def convolutional_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

In [52]:
# Normal full layer
def normal_full_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,W)+b

In [53]:
# Placeholders
x = tf.placeholder(tf.float32,shape=[None,784])

In [54]:
y_true = tf.placeholder(tf.float32,shape=[None,10])

In [55]:
# Layers
x_image = tf.reshape(x,[-1,28,28,1])

In [56]:
convo_1 = convolutional_layer(x_image,shape=[5,5,1,32])
conv_1_pooling = max_pooling(convo_1)

In [57]:
convo_2 = convolutional_layer(conv_1_pooling,shape=[5,5,32,64])
conv_2_pooling = max_pooling(convo_2)

In [58]:
convo_2_flat = tf.reshape(conv_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [59]:
# Dropout
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [60]:
y_pred = normal_full_layer(full_one_dropout,10)

In [61]:
# Loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true,logits=y_pred))

In [71]:
optimizer = tf.train.AdadeltaOptimizer(learning_rate=0.01)

In [72]:
train = optimizer.minimize(cross_entropy)

In [73]:
init = tf.global_variables_initializer()

In [74]:
steps = 1000

In [75]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50)
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5} )
        
        if i%100 ==0:
            print("On step:{}".format(i))
            print("Accuracy: ")
            matches = tf.equal(tf.arg_max(y_pred,1),tf.arg_max(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,\
                                          hold_prob:1.0}))
            print('\n')

On step:0
Accuracy: 
0.0919


On step:100
Accuracy: 
0.0868


On step:200
Accuracy: 
0.1954


On step:300
Accuracy: 
0.3141


On step:400
Accuracy: 
0.409


On step:500
Accuracy: 
0.5043


On step:600
Accuracy: 
0.5702


On step:700
Accuracy: 
0.6226


On step:800
Accuracy: 
0.6624


On step:900
Accuracy: 
0.6933


